## The Rosetta E scores in ben_ddg.ipynb are scores for the stability of the complex, but do not tell us the change in free energy of Binding between the mutant Spike and Ace2

To find the change in free energy of binding, we need to score the proteins individually (the mutant spike and ace2), then compare these scores to the score of the complex.  
Chris Mathy wrote a function that returns this change in free energy between the individual proteins and the complex they form (get_interface_energy).

In [1]:
import os
import re
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

import pyrosetta
from pyrosetta import *
pyrosetta.init('''
               -ex1
               -ex2
               -score:weights ref2015_cart
               -use_input_sc
               -ignore_unrecognized_res           
               ''')
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.pack.task import operation
from pyrosetta.rosetta.core.select import residue_selector

scorefxn = create_score_function('ref2015_cart')

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release 2021.11+release.e9f47978df05d6d2de96236c2d949ecf9436018a 2021-03-19T09:06:15] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release r276 2021.11+release.e9f47978df0 e9f47978df05d6d2de96236c2d949ecf9436018a http://www.pyrosetta.org 2021-03-19T09:06:15
core.init: {0} command: PyRosetta -ex1 -ex2 -score:weights ref2015_cart -use_input_sc -ignore_unrecognized_res -database /opt/anaconda3/envs/proteindesign/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=796381290 seed_offset=0 real_seed=796381290 thread_index=0
basic.random.init_random_generator: {0} RandomGenera

In [2]:
spike_ace2_clean = '../Kevin/6M17_clean.pdb'

pose = pose_from_pdb(spike_ace2_clean)
pose.pdb_info().name('6M17_clean')

core.import_pose.import_pose: {0} File '../Kevin/6M17_clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue ARG:CtermProteinFull 748
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue LEU:CtermProteinFull 931
core.conformation.Conformation: {0} Found disulfide between residues 324 341
core.conformation.Conformation: {0} current variant for 324 CYS
core.conformation.Conformation: {0} current variant for 341 CYS
core.conformation.Conformation: {0} current variant for 324 CYD
core.conformation.Conformation: {0} current variant for 341 CYD
core.conformation.Conformation: {0} Found disulfide between residues 510 522
core.conformation.Conformation: {0} current variant for 510 CYS
core.conformation.Conformation: {0} current variant for 522 CYS
core.conformation.Conformation: {0} current variant for 510 CYD
core.conformation.Conformation: {0} current variant for 522 CYD
core.confo

In [3]:
spike_ace2_relaxed = './6M17_relaxed.pdb'

pose_relaxed = pose_from_pdb(spike_ace2_relaxed)
pose_relaxed.pdb_info().name('6M17_relaxed')

core.import_pose.import_pose: {0} File './6M17_relaxed.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} Found disulfide between residues 324 341
core.conformation.Conformation: {0} current variant for 324 CYS
core.conformation.Conformation: {0} current variant for 341 CYS
core.conformation.Conformation: {0} current variant for 324 CYD
core.conformation.Conformation: {0} current variant for 341 CYD
core.conformation.Conformation: {0} Found disulfide between residues 792 845
core.conformation.Conformation: {0} current variant for 792 CYS
core.conformation.Conformation: {0} current variant for 845 CYS
core.conformation.Conformation: {0} current variant for 792 CYD
core.conformation.Conformation: {0} current variant for 845 CYD
core.conformation.Conformation: {0} Found disulfide between residues 893 901
core.conformation.Conformation: {0} current variant for 893 CYS
core.conformation.Conformation: {0} current variant for 901 CYS
core.conformation.Conforma

In [4]:
## From Chris Mathy: Function to measure interface E
## Separates two chains far apart, then recalculates Rosetta E

# Leave pack_input and pack_separated as False
# Might just introduce inconsistencies

# If dump_pdb_filename='.pdb', then the pdb file it spits 

def get_interface_energy(pose, scorefxn, interface,
                  pack_input=False, pack_separated=False,
                  dump_pdb_filename=None):
    '''
    pose: the complex to be scored
    scorefxn: a rosetta score function
    interface: str listing the chains forming the interface, i.e. A_B
    pack_input: bool, whether to re-pack the input structure
    pack_separated: bool, whether to re-pack the structures after seperation
    dump_pdb: str, path to file to dump the pdb with the interface analyzer scores in the file. Defaults to NULL, no dump.
    '''
    p = pose.clone()
    ia_mover = pyrosetta.rosetta.protocols.analysis.InterfaceAnalyzerMover()
    ia_mover.set_scorefunction(scorefxn)
    ia_mover.set_interface(interface)
    ia_mover.set_pack_input(pack_input)
    ia_mover.set_pack_separated(pack_separated)
    ia_mover.apply(p)
    ia_mover.add_score_info_to_pose(p)
    if (dump_pdb_filename):
        p.dump_pdb(dump_pdb_filename)
    return ia_mover.get_separated_interface_energy()

In [6]:
wt_dG_binding = 0

pose = pose_from_pdb('6M17_relaxed.pdb')
scorefxn = create_score_function('ref2015_cart')
interface_energy = get_interface_energy(pose=pose, scorefxn=scorefxn, interface='D_F', \
                                        dump_pdb_filename='WT_interface_energy.pdb')
wt_dG_binding = interface_energy

core.import_pose.import_pose: {0} File '6M17_relaxed.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} Found disulfide between residues 324 341
core.conformation.Conformation: {0} current variant for 324 CYS
core.conformation.Conformation: {0} current variant for 341 CYS
core.conformation.Conformation: {0} current variant for 324 CYD
core.conformation.Conformation: {0} current variant for 341 CYD
core.conformation.Conformation: {0} Found disulfide between residues 792 845
core.conformation.Conformation: {0} current variant for 792 CYS
core.conformation.Conformation: {0} current variant for 845 CYS
core.conformation.Conformation: {0} current variant for 792 CYD
core.conformation.Conformation: {0} current variant for 845 CYD
core.conformation.Conformation: {0} Found disulfide between residues 893 901
core.conformation.Conformation: {0} current variant for 893 CYS
core.conformation.Conformation: {0} current variant for 901 CYS
core.conformation.Conformati

In [7]:
wt_dG_binding

-58.257234320633415

In [8]:
muts = ['A', 'D', 'F', 'G', 'I', 'L', 'M', 'P', 'V', 'W', 'Y']
mut_dGs_binding = {}

for one_letter_code in muts:
    pose = pose_from_pdb('N501' + one_letter_code + '_relaxed.pdb')
    scorefxn = create_score_function('ref2015_cart')
    interface_energy = get_interface_energy(pose=pose, scorefxn=scorefxn, interface='D_F', \
                                            dump_pdb_filename='N501' + one_letter_code + '_interface_energy.pdb')
    mut_dGs_binding[one_letter_code] = interface_energy
    

core.import_pose.import_pose: {0} File 'N501A_relaxed.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} Found disulfide between residues 324 341
core.conformation.Conformation: {0} current variant for 324 CYS
core.conformation.Conformation: {0} current variant for 341 CYS
core.conformation.Conformation: {0} current variant for 324 CYD
core.conformation.Conformation: {0} current variant for 341 CYD
core.conformation.Conformation: {0} Found disulfide between residues 792 845
core.conformation.Conformation: {0} current variant for 792 CYS
core.conformation.Conformation: {0} current variant for 845 CYS
core.conformation.Conformation: {0} current variant for 792 CYD
core.conformation.Conformation: {0} current variant for 845 CYD
core.conformation.Conformation: {0} Found disulfide between residues 893 901
core.conformation.Conformation: {0} current variant for 893 CYS
core.conformation.Conformation: {0} current variant for 901 CYS
core.conformation.Conformat

protocols.analysis.InterfaceAnalyzerMover: {0} [ WARNING ] In InterfaceAnalyzerMover, calculator Sasa_1 already exists, this is hopefully correct for your purposes
protocols.analysis.InterfaceAnalyzerMover: {0} [ WARNING ] In InterfaceAnalyzerMover, calculator InterfaceNeighborDefinition_1 already exists, this is hopefully correct for your purposes
protocols.analysis.InterfaceAnalyzerMover: {0} [ WARNING ] In InterfaceAnalyzerMover, calculator InterfaceDeltaEnergetics_1 already exists, this is hopefully correct for your purposes
protocols.analysis.InterfaceAnalyzerMover: {0} [ WARNING ] In InterfaceAnalyzerMover, calculator NumberHBonds_1 already exists, this is hopefully correct for your purposes
protocols.analysis.InterfaceAnalyzerMover: {0} [ WARNING ] In InterfaceAnalyzerMover, calculator BuriedUnsatisfiedPolars_1 already exists, this is hopefully correct for your purposes
protocols.analysis.InterfaceAnalyzerMover: {0} Making an interface set with fixed chains
protocols.analysis.In

core.conformation.Conformation: {0} current variant for 510 CYD
core.conformation.Conformation: {0} current variant for 522 CYD
core.conformation.Conformation: {0} current variant for 510 CYD
core.conformation.Conformation: {0} current variant for 522 CYD
core.conformation.Conformation: {0} Found disulfide between residues 792 845
core.conformation.Conformation: {0} current variant for 792 CYD
core.conformation.Conformation: {0} current variant for 845 CYD
core.conformation.Conformation: {0} current variant for 792 CYD
core.conformation.Conformation: {0} current variant for 845 CYD
core.conformation.Conformation: {0} Found disulfide between residues 893 901
core.conformation.Conformation: {0} current variant for 893 CYD
core.conformation.Conformation: {0} current variant for 901 CYD
core.conformation.Conformation: {0} current variant for 893 CYD
core.conformation.Conformation: {0} current variant for 901 CYD
protocols.analysis.InterfaceAnalyzerMover: {0} Calculating dSASA
protocols.ana

protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 1 and 2
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 2 and 1
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 1 and 2
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 1 and 2
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 2 and 1
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 1 and 2
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 1 and 2
protocols.analysis.InterfaceAnalyzerMover: {0} Computing delta unsat polar residues...
protocols.analysis.InterfaceAnalyzerMover: {0} Computing Shape Complementarity Score...
protocols.analysis.InterfaceAnalyzerMover: {0} Upstream chain(s) numbers: 1,
protocols.analysis.InterfaceAnalyzerMover: {0} Downstream chain(s) numbers: 2,
core.import_pose.import_pose: {0} File 'N501I_relaxed.pdb' automatically determined to b

core.conformation.Conformation: {0} current variant for 845 CYD
core.conformation.Conformation: {0} Found disulfide between residues 893 901
core.conformation.Conformation: {0} current variant for 893 CYS
core.conformation.Conformation: {0} current variant for 901 CYS
core.conformation.Conformation: {0} current variant for 893 CYD
core.conformation.Conformation: {0} current variant for 901 CYD
core.conformation.Conformation: {0} Found disulfide between residues 510 522
core.conformation.Conformation: {0} current variant for 510 CYS
core.conformation.Conformation: {0} current variant for 522 CYS
core.conformation.Conformation: {0} current variant for 510 CYD
core.conformation.Conformation: {0} current variant for 522 CYD
protocols.analysis.InterfaceAnalyzerMover: {0} Using explicit constructor
protocols.analysis.InterfaceAnalyzerMover: {0} Using interface constructor
protocols.analysis.InterfaceAnalyzerMover: {0} [ WARNING ] In InterfaceAnalyzerMover, calculator Sasa_1 already exists, t

protocols.analysis.InterfaceAnalyzerMover: {0} Making an interface set with fixed chains
protocols.analysis.InterfaceAnalyzerMover: {0} Interface set residues total: 74
core.energy_methods.CartesianBondedEnergy: {0} Creating new peptide-bonded energy container (931)
core.conformation.Conformation: {0} Found disulfide between residues 324 341
core.conformation.Conformation: {0} current variant for 324 CYD
core.conformation.Conformation: {0} current variant for 341 CYD
core.conformation.Conformation: {0} current variant for 324 CYD
core.conformation.Conformation: {0} current variant for 341 CYD
core.conformation.Conformation: {0} Found disulfide between residues 510 522
core.conformation.Conformation: {0} current variant for 510 CYD
core.conformation.Conformation: {0} current variant for 522 CYD
core.conformation.Conformation: {0} current variant for 510 CYD
core.conformation.Conformation: {0} current variant for 522 CYD
core.conformation.Conformation: {0} Found disulfide between residue

protocols.analysis.InterfaceAnalyzerMover: {0} Calculating dSASA
protocols.analysis.InterfaceAnalyzerMover: {0} Calculating per-res dSASA data
protocols.analysis.InterfaceAnalyzerMover: {0} included_nres: 931
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015_cart
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 2 and 1
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 2 and 1
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 2 and 1
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 1 and 2
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 2 and 1
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 2 and 1
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 2 and 1
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond between chains: 1 and 2
protocols.analysis.InterfaceAnalyzerMover: {0} Found Hbond b

protocols.analysis.InterfaceAnalyzerMover: {0} Upstream chain(s) numbers: 1,
protocols.analysis.InterfaceAnalyzerMover: {0} Downstream chain(s) numbers: 2,
core.import_pose.import_pose: {0} File 'N501W_relaxed.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} Found disulfide between residues 324 341
core.conformation.Conformation: {0} current variant for 324 CYS
core.conformation.Conformation: {0} current variant for 341 CYS
core.conformation.Conformation: {0} current variant for 324 CYD
core.conformation.Conformation: {0} current variant for 341 CYD
core.conformation.Conformation: {0} Found disulfide between residues 792 845
core.conformation.Conformation: {0} current variant for 792 CYS
core.conformation.Conformation: {0} current variant for 845 CYS
core.conformation.Conformation: {0} current variant for 792 CYD
core.conformation.Conformation: {0} current variant for 845 CYD
core.conformation.Conformation: {0} Found disulfide between residues 893 901

protocols.analysis.InterfaceAnalyzerMover: {0} Using interface constructor
protocols.analysis.InterfaceAnalyzerMover: {0} [ WARNING ] In InterfaceAnalyzerMover, calculator Sasa_1 already exists, this is hopefully correct for your purposes
protocols.analysis.InterfaceAnalyzerMover: {0} [ WARNING ] In InterfaceAnalyzerMover, calculator InterfaceNeighborDefinition_1 already exists, this is hopefully correct for your purposes
protocols.analysis.InterfaceAnalyzerMover: {0} [ WARNING ] In InterfaceAnalyzerMover, calculator InterfaceDeltaEnergetics_1 already exists, this is hopefully correct for your purposes
protocols.analysis.InterfaceAnalyzerMover: {0} [ WARNING ] In InterfaceAnalyzerMover, calculator NumberHBonds_1 already exists, this is hopefully correct for your purposes
protocols.analysis.InterfaceAnalyzerMover: {0} [ WARNING ] In InterfaceAnalyzerMover, calculator BuriedUnsatisfiedPolars_1 already exists, this is hopefully correct for your purposes
protocols.analysis.InterfaceAnalyze

In [9]:
mut_dGs_binding

{'A': -58.52676862475619,
 'D': -59.489615171005426,
 'F': -61.453709624119256,
 'G': -58.324729388232754,
 'I': -59.422239995230484,
 'L': -58.436606204946656,
 'M': -59.01687305632777,
 'P': -58.14437117572015,
 'V': -59.976847086154066,
 'W': -56.423550596200585,
 'Y': -61.500239417626744}

In [11]:
for mut in mut_dGs_binding:
    print('%s %f' % (mut, mut_dGs_binding[mut] - wt_dG_binding))

A -0.269534
D -1.232381
F -3.196475
G -0.067495
I -1.165006
L -0.179372
M -0.759639
P 0.112863
V -1.719613
W 1.833684
Y -3.243005
